In [ ]:
from file_search import search_files

In [ ]:
search_directory = r'C:\Users\{user}\Documents'
exclude_folders = ['cache', 'Fax', 'Auto-Services Temp']
extensions = ['.xlsx', '.xls', '.docx']
size_limit = 100

patterns = '''
string1
string1
string1
'''

results = search_files(pattern=patterns,
                       top_dir=search_directory,
                       extensions=extensions,
                       exclude_folders=exclude_folders,
                       size_limit=size_limit,
                       print_errors=True)